In [84]:
import pandas as pd
import os
print(os.getcwd())
import pandas as pd
print(pd.__version__)
!python --version
!jupyter --version

# Load data from the input folder
enrollment_data = pd.read_csv("../data/input/CPSC_Enrollment_2015_01/CPSC_Enrollment_Info_2015_01.csv")
service_area_data = pd.read_csv("../data/input/MA_Cnty_SA_2015_01/MA_Cnty_SA_2015_01.csv")
contract_data = pd.read_csv("../data/input/CPSC_Enrollment_2015_01/CPSC_Contract_Info_2015_01.csv", encoding='latin1')
pd.set_option('display.max_columns', None)



# Display the datasets
print(enrollment_data.head())
print(service_area_data.head())


/Users/ryanscholte/Desktop/GitHub/HW1/submission1
2.2.3
Python 3.10.16
Selected Jupyter core packages...
IPython          : 8.30.0
ipykernel        : 6.29.5
ipywidgets       : not installed
jupyter_client   : 8.6.3
jupyter_core     : 5.7.2
jupyter_server   : not installed
jupyterlab       : not installed
nbclient         : not installed
nbconvert        : not installed
nbformat         : 5.10.4
notebook         : not installed
qtconsole        : not installed
traitlets        : 5.14.3
  Contract Number  Plan ID  SSA State County Code  FIPS State County Code  \
0           E0654      801                  58160                     NaN   
1           E0654      801                  61000                     NaN   
2           E0654      801                  96001                     NaN   
3           E0654      801                   2275                     NaN   
4           E0654      801                   2198                     NaN   

  State County Enrollment  
0   NaN    NaN     

/var/folders/mn/l2nrwsxn24g6ywz6ygh2fxp40000gn/T/ipykernel_47678/4202740048.py:11: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  service_area_data = pd.read_csv("../data/input/MA_Cnty_SA_2015_01/MA_Cnty_SA_2015_01.csv")


In [85]:
print(enrollment_data.columns)
print(contract_data.columns)
print(service_area_data.columns)


Index(['Contract Number', 'Plan ID', 'SSA State County Code',
       'FIPS State County Code', 'State', 'County', 'Enrollment'],
      dtype='object')
Index(['Contract ID', 'Plan ID', 'Organization Type', 'Plan Type',
       'Offers Part D', 'SNP Plan', 'EGHP', 'Organization Name',
       'Organization Marketing Name', 'Plan Name', 'Parent Organization',
       'Contract Effective Date'],
      dtype='object')
Index(['Contract ID', 'Organization Name', 'Organization Type', 'Plan Type',
       'Partial', 'EGHP', 'SSA', 'FIPS', 'County', 'State', 'Notes'],
      dtype='object')


In [86]:
#rename Contract Number to Contract ID , SSA State county code to SSA, and FIPS state county code to FIPS in enrollment data
enrollment_data.rename(columns={'Contract Number': 'Contract ID', 'SSA State County Code': 'SSA', 'FIPS State County Code': 'FIPS'}, inplace=True)

 # merge enrollment data with contract data on 'Contract ID' and 'Plan ID'
merged_data = pd.merge(enrollment_data, contract_data, on=['Contract ID', 'Plan ID'], how='left')

# #show me the merged data head
print(merged_data.head())

  Contract ID  Plan ID    SSA  FIPS State County Enrollment  \
0       E0654      801  58160   NaN   NaN    NaN          *   
1       E0654      801  61000   NaN   NaN    NaN          *   
2       E0654      801  96001   NaN   NaN    NaN          *   
3       E0654      801   2275   NaN   NaN    NaN          *   
4       E0654      801   2198   NaN   NaN    NaN          *   

                         Organization Type  \
0  Employer/Union Only Direct Contract PDP   
1  Employer/Union Only Direct Contract PDP   
2  Employer/Union Only Direct Contract PDP   
3  Employer/Union Only Direct Contract PDP   
4  Employer/Union Only Direct Contract PDP   

                                 Plan Type Offers Part D SNP Plan EGHP  \
0  Employer/Union Only Direct Contract PDP           Yes       No  Yes   
1  Employer/Union Only Direct Contract PDP           Yes       No  Yes   
2  Employer/Union Only Direct Contract PDP           Yes       No  Yes   
3  Employer/Union Only Direct Contract PDP      

In [87]:
# Convert Plan ID to same type in both dataframes
enrollment_data['Plan ID'] = enrollment_data['Plan ID'].astype(float)
contract_data['Plan ID'] = contract_data['Plan ID'].astype(float)
merged_data['Plan ID'] = merged_data['Plan ID'].astype(float)
# Convert Contract ID to string type in both
enrollment_data['Contract ID'] = enrollment_data['Contract ID'].astype(str)
contract_data['Contract ID'] = contract_data['Contract ID'].astype(str)
merged_data['Contract ID'] = merged_data['Contract ID'].astype(str)

# Merge 
merged_all = pd.merge(merged_data, 
                                   service_area_data,
                                   on=['Contract ID', 'SSA', 'FIPS', 'State', 'County'],
                                   how='inner')

# check the merged data
print(merged_all.head())

  Contract ID  Plan ID    SSA     FIPS State    County Enrollment  \
0       H0022      1.0  36110  39023.0    OH     Clark        598   
1       H0022      1.0  36170  39035.0    OH  Cuyahoga       2810   
2       H0022      1.0  36260  39051.0    OH    Fulton        112   
3       H0022      1.0  36280  39055.0    OH    Geauga         48   
4       H0022      1.0  36290  39057.0    OH    Greene        528   

  Organization Type_x                        Plan Type_x Offers Part D  \
0                Demo  Medicare-Medicaid Plan HMO/HMOPOS           Yes   
1                Demo  Medicare-Medicaid Plan HMO/HMOPOS           Yes   
2                Demo  Medicare-Medicaid Plan HMO/HMOPOS           Yes   
3                Demo  Medicare-Medicaid Plan HMO/HMOPOS           Yes   
4                Demo  Medicare-Medicaid Plan HMO/HMOPOS           Yes   

  SNP Plan EGHP_x                  Organization Name_x  \
0       No     No  BUCKEYE COMMUNITY HEALTH PLAN, INC.   
1       No     No  BUCKE

In [88]:
#rename column "Plan Type_x" to "Plan Type", EGHP_x to EGHP
merged_all.rename(columns={'Plan Type_x': 'Plan Type', 'EGHP_x': 'EGHP'}, inplace=True)

# Count the number of plans for each plan type
plan_counts = merged_all.groupby('Plan Type').size().reset_index(name='Plan Count')
print(plan_counts)

                           Plan Type  Plan Count
0                          1876 Cost        5527
1                         HMO/HMOPOS      453802
2                          Local PPO      701380
3                                MSA        6502
4  Medicare-Medicaid Plan HMO/HMOPOS         859
5                      National PACE         649
6                               PFFS       12596
7                       Regional PPO       16048


In [89]:
#make merged data 2 set which drops all special needs plans (SNP), employer group plans (eghp), and all “800-series”
# in "SNP Plan" column drop is says yes, in "EGHP" coloumn if says yes drop it, in "Plan ID" column between 800-900 drop it
merged_all2 = merged_all[~merged_all['SNP Plan'].str.contains('Yes', na=False) &
                           ~merged_all['EGHP'].str.contains('Yes', na=False) &
                           ~merged_all['Plan ID'].between(800, 900, inclusive="left")]
print(merged_all2.head())

  Contract ID  Plan ID    SSA     FIPS State    County Enrollment  \
0       H0022      1.0  36110  39023.0    OH     Clark        598   
1       H0022      1.0  36170  39035.0    OH  Cuyahoga       2810   
2       H0022      1.0  36260  39051.0    OH    Fulton        112   
3       H0022      1.0  36280  39055.0    OH    Geauga         48   
4       H0022      1.0  36290  39057.0    OH    Greene        528   

  Organization Type_x                          Plan Type Offers Part D  \
0                Demo  Medicare-Medicaid Plan HMO/HMOPOS           Yes   
1                Demo  Medicare-Medicaid Plan HMO/HMOPOS           Yes   
2                Demo  Medicare-Medicaid Plan HMO/HMOPOS           Yes   
3                Demo  Medicare-Medicaid Plan HMO/HMOPOS           Yes   
4                Demo  Medicare-Medicaid Plan HMO/HMOPOS           Yes   

  SNP Plan EGHP                  Organization Name_x  \
0       No   No  BUCKEYE COMMUNITY HEALTH PLAN, INC.   
1       No   No  BUCKEYE COM

In [90]:
# Count the number of plans for each plan type for merged all 2 
plan_counts2 = merged_all2.groupby('Plan Type').size().reset_index(name='Plan Count')
print(plan_counts2)


                           Plan Type  Plan Count
0                          1876 Cost        4905
1                         HMO/HMOPOS       19449
2                          Local PPO       14618
3                                MSA         222
4  Medicare-Medicaid Plan HMO/HMOPOS         859
5                      National PACE         649
6                               PFFS        3179
7                       Regional PPO        7375


In [91]:
#show values for enrollment column in merged all 2
print(merged_all2['Enrollment'].value_counts())
# sum of enrollment column in merged all 2
# print(merged_all2['Enrollment'].sum())




Enrollment
*       26511
11        439
13        436
12        430
14        409
        ...  
3081        1
5425        1
5543        1
3928        1
1089        1
Name: count, Length: 2637, dtype: int64


In [92]:
#remove non numeric values from enrollment column
merged_all2['Enrollment'] = pd.to_numeric(merged_all2['Enrollment'], errors='coerce')
#average enrollments for each plan type
average_enrollment = merged_all2.groupby('Plan Type')['Enrollment'].mean().reset_index(name='Average Enrollment')
print(average_enrollment)



                           Plan Type  Average Enrollment
0                          1876 Cost          229.650200
1                         HMO/HMOPOS          871.225487
2                          Local PPO          311.062571
3                                MSA          107.792793
4  Medicare-Medicaid Plan HMO/HMOPOS          753.030303
5                      National PACE          144.487805
6                               PFFS          125.366225
7                       Regional PPO          201.502988


/var/folders/mn/l2nrwsxn24g6ywz6ygh2fxp40000gn/T/ipykernel_47678/2313252136.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  merged_all2['Enrollment'] = pd.to_numeric(merged_all2['Enrollment'], errors='coerce')
